<a href="https://colab.research.google.com/github/ElPapi42/AgeClassifier/blob/master/AgeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.display import clear_output
!pip install --upgrade tensorflow-gpu
!pip install --upgrade tqdm
!pip install --upgrade pillow
!pip install git+https://github.com/Jwink3101/parmapper
clear_output()

In [0]:
#Imports
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from parmapper import parmap
import sys

## Data Download

In [0]:
#Downloads and extract Dataset to local, wait for download, i dont want to put a progress bar here sorry
#You can run this on google colab for get faster downloads speeds
import os
import tarfile
import requests
from tqdm import tqdm

folder_path = "./Datasets"

wiki_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"
wiki_path = folder_path + '/wiki.tar.gz'

imdb_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar"
imdb_path = folder_path + '/imdb.tar.gz'

#Create Dataset folder if not exists
if(not os.path.exists(folder_path)):
  os.mkdir(folder_path)

#download wikipedia faces dataset
if(not(os.path.exists(wiki_path) and os.path.isfile(wiki_path))):
  resp = requests.get(wiki_url, stream=True)

  total_size = int(resp.headers.get('content-length', 0))
  block_size = 16384
  t=tqdm(total=total_size, unit='iB', unit_scale=True)
  
  with open(wiki_path, "wb") as f:
    for data in resp.iter_content(block_size):
      t.update(len(data))
      f.write(data)
    t.close()
    f.close()

    if total_size != 0 and t.n != total_size:
      print("Wiki Faces Download Error")

#download imdb faces dataset
if(not(os.path.exists(imdb_path) and os.path.isfile(imdb_path))):
  resp = requests.get(imdb_url, stream=True)

  total_size = int(resp.headers.get('content-length', 0))
  block_size = 16384
  t=tqdm(total=total_size, unit='iB', unit_scale=True)
  
  with open(imdb_path, "wb") as f:
    for data in resp.iter_content(block_size):
      t.update(len(data))
      f.write(data)
    t.close()
    f.close()

    if total_size != 0 and t.n != total_size:
      print("Imdb Faces Download Error")

100%|██████████| 811M/811M [00:23<00:00, 34.0MiB/s]
100%|██████████| 7.01G/7.01G [06:23<00:00, 18.3MiB/s]


In [0]:
#Extract Datasets

#Wikipedia Dataset
tar = tarfile.open(wiki_path, "r")
tar.extractall(folder_path)
tar.close()
os.remove("./Datasets/wiki_crop/wiki.mat")

#IMDB Dataset
tar = tarfile.open(imdb_path, "r")
tar.extractall(folder_path)
tar.close()
os.remove("./Datasets/imdb_crop/imdb.mat")

In [0]:
def preproc(filepath):
  try:
    #Check if the image is corrupted
    Image.open(filepath)

    filename = filepath[:-4].split("/")[-1]
    folder = filepath[:-4].split("/")[1]
    file_data = filename.split("_")

    if(folder == "wiki_crop"):
      id = "w" + file_data[0]
      born = int(file_data[1].split("-")[0])
    else:
      id = "i" + file_data[0][2:]
      born = int(file_data[2].split("-")[0])

    taken = int(file_data[-1])
    age = taken - born
    age = str(age)

    return [id, age, filepath]
  except OSError:
    os.remove(filepath)

#Find urls
root = "./Datasets"
folders = os.listdir(root)

if(not ("dataset.csv" in folders)):

  folders.remove("imdb.tar.gz")
  folders.remove("wiki.tar.gz")

  urls = list()

  for folder in folders:
    subfolders = os.listdir("{root}/{folder}".format(root=root, folder=folder))
    for subfolder in subfolders:
      files = os.listdir("{root}/{folder}/{subfolder}".format(root=root, folder=folder, subfolder=subfolder))
      for file in files:
        path = "{root}/{folder}/{subfolder}/{file}".format(root=root, folder=folder, subfolder=subfolder, file=file)
        urls.append(path)

  #Extract ids and age from images file names
  processed = list(parmap(preproc, urls, progress=True))
  faces_df = pd.DataFrame(processed, columns=["id", "age", "url"])
  faces_df.to_csv(root + "/dataset.csv", index=False)

0it [00:00, ?it/s]


## Exploratory Data analysis and Cleaning

In [0]:
#Load csv
faces_df = pd.read_csv("./Datasets/dataset.csv")

#Shuffle
faces_df = faces_df.sample(frac=1.0)
faces_df.head()

,0,1
170536,NaN,NaN
132941,NaN,NaN
370583,NaN,NaN
435864,NaN,NaN
405408,NaN,NaN


In [0]:
faces_df.info()

In [0]:
#Lets see some examples
plt.figure(figsize=(10,10))
for i, path in enumerate(faces_df["url"].iloc[:9]):
    img = tf.keras.preprocessing.image.load_img(path)
    plt.subplot(3,3,i+1)
    plt.imshow(img)
plt.show()

In [0]:
#Check descriptive statistics
faces_df.describe()

High Precense of outliers, lets fix that

In [0]:
#Clear Dataset from outliers
age_up_0 = faces_df["age"] > 0
age_down_75 = faces_df["age"] < 75
faces_df = faces_df[age_up_0 & age_down_75]
faces_df.describe()

In [0]:
#Check for how much unique people is here
id_count = faces_df["personId"].nunique()
"The Dataset contains {count} unique persons across its images".format(count=id_count)

In [0]:
#Lets see the distribution of the age across our dataset
sns.distplot(faces_df["age"], bins=75)

## Dataset Preparation

In [0]:
#Split dataset between train, eval and test
full_df = faces_df
eval_df = full_df.sample(frac=0.05)
full_df = full_df.drop(eval_df.index)
test_df = full_df.sample(frac=0.05)
train_df = full_df.drop(test_df.index)

In [0]:
#Store number of datapoints in each dataset
train_count = train_df.shape[0]
eval_count = eval_df.shape[0]
test_count = test_df.shape[0]
print("Train samples: " + str(train_count))
print("Evaluation samples: " + str(eval_count))
print("Test samples: " + str(test_count))

In [0]:
#Verify the distribution of the three splits
sns.distplot(a=train_df["age"], bins=75).set_title("Users Distribution")
sns.distplot(a=eval_df["age"], bins=75)
sns.distplot(a=test_df["age"], bins=75)

plt.show()

## Dataset Creation

In [0]:
#Generator funcion
tf.train.

In [0]:
#Create Datasets for train, evaluation and testing
train_ds = tf.data.Dataset.from_tensor_slices().shuffle(4196)
eval_ds = tf.data.Dataset.from_tensor_slices().batch(eval_count)
test_ds = tf.data.Dataset.from_tensor_slices().batch(test_count)